In [1]:
import os
import json
import sqlite3
import pandas as pd
import shutil
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
data_igtao = datetime.now().strftime('%Y-%m-%d')

caminho_backup = '/workspaces/projeto_ing/INGESTION/app/src/backp/'
ctrl_ing = "/workspaces/projeto_ing/INGESTION/app/src/archiving_ctrl/arquivo_controle.txt"
landing = '/workspaces/projeto_ing/INGESTION/app/src/landing/'
layouts = '/workspaces/projeto_ing/INGESTION/app/src/tmp/layouts/'

In [2]:
read_landing = os.listdir(landing)
print("Conteúdos do diretório 'landing':", read_landing)
read_layouts = os.listdir(layouts)
print("Conteúdos do diretório 'layouts':", read_layouts)
print('===============================================')

Conteúdos do diretório 'landing': ['resultado_teste2_20242306_215405.txt']
Conteúdos do diretório 'layouts': ['teste']


In [6]:
for item in read_layouts:
    accept = os.path.join(layouts, item)
    if os.path.isdir(accept):
        print('========================')
        print(f'item encontrado no diretório: {item}')
        subconteudos = os.listdir(accept)
        print('========================')
        print(f' conteudo: {subconteudos}')
        
        caminho_estrutura_json = os.path.join(accept, 'estrutura.json')
        if os.path.exists(caminho_estrutura_json):
            print(f'Encontrado estrutura.json em {item}')
            
            with open(caminho_estrutura_json, 'r') as f:
                    config_estrutura = json.load(f)
                    print("Configuração estrutura:", config_estrutura)
                    
                    for estr in config_estrutura:
                        nm_arq = estr['nome_arquivo']
                        key = estr['chave']['palavra_chave']
                        banco = estr['chave']['banco']
                        tabela = estr['chave']['tabela']
                        print('=======================')
                        print(estr)
                        print(nm_arq)
                        print(key)
                        print(banco)
                        print(tabela)
                        print('=======================')
                        
                        with open(ctrl_ing, 'r+') as controle_ingestao:
                            conteudo_load = controle_ingestao.readlines()
                            print(f'Conteúdo do arquivo de controle de ingestão: {conteudo_load}')
                            
                        for root, dirs, lista in os.walk(landing):
                            print(f'Procurando em {root}...')
                            for nome_arquivo in lista:
                                print(f'Verificando arquivo: {nome_arquivo}')
                                if key in nome_arquivo:
                                    caminho_completo = os.path.join(root, nome_arquivo)
                                    print(f'Arquivo contendo a chave encontrado: {caminho_completo}')
                                    sav = os.path.basename(caminho_completo)
                                    print(f'Conteúdo a gravar: {sav}')
                                    
                                    #AJUSTAR O CONTROLE, NA HORA QUE ELE LE O CONTROLE SE ENCONTRAR O ARQUIVO COM O MESMO NOME E TIMESTAMP,
                                    #DEVE IGNORAR O MESMO, SÓ DEVE PROCESSEGUIR COM A INGESTÃO CASO O ARQUIVO NÃO SE ENCONTRAR NO CONTROLE DE INGESTÃO
                                    #NO MOMENTO O CÓDIGO ESTÁ IGNORANDO ESTÁ AÇÃO E INGERINDO O MESMO DADO.
                                    #VERIFICAR SE É NA HORA DE GRAVAR OU ENTÃO NA HORA DE REALIZAR A LEITURA DO ARQUIVO DE CONTROLE.
                                    
                                    # if sav + '\n' not in conteudo_load:
                                    #     controle_ingestao.write(f"{sav}\n")
                                    #     print(f'Dado salvo: {sav}')
                                    #     controle_ingestao.flush()
                                    #     print('=================================')
                                    if sav + '\n' not in conteudo_load:
                                        with open(ctrl_ing, 'a') as controle_ingestao:
                                            controle_ingestao.write(f"{sav}\n")
                                        print(f'Dado salvo: {sav}')
                                        print('=================================')
                                    

                                    
                                    parte_util = sav.split('_')[1]
                                    if parte_util == key:
                                        print('Chave encontrada')
                                        print(f'Separando nome do arquivo: {parte_util}')
                                        caminho_tabela_json = os.path.join(accept, f'{tabela}.json')
                                        print(f'local para criar tabela: {caminho_tabela_json}')
                                        if os.path.exists(caminho_tabela_json):
                                            with open(caminho_tabela_json, 'r') as f:     
                                                config_colunas = json.load(f)
                                            print(config_colunas)
                                            print(f'resultado configuracao colunas: {config_colunas}')
                                            print('===============================')
                                            for j in config_colunas['colunas']:
                                                print(f'iterando config: {j}')
                                            
                                            if caminho_completo.endswith('.txt'):
                                                df = pd.read_csv(caminho_completo, delimiter=';', skiprows=[0], names=[col['nome'] for col in config_colunas['colunas'] if 'nome' in col])
                                                print(f'dados a carregar:{df}')
                                                print('estrutura de colunas txt:')
                                                print(df.columns)
                                                print('=====================')
                                            elif caminho_completo.endswith('.csv'):
                                                df = pd.read_csv(caminho_completo, skiprows=[0], names=[col['nome'] for col in config_colunas['colunas'] if 'nome' in col])
                                                print(f'dados a carregar:{df}')
                                                print('estrutura de colunas csv:')
                                                print(df.columns)
                                                print('=====================')
                                            elif caminho_completo.endswith('.bin'):
                                                # Lógica para arquivos binários
                                                df = pd.DataFrame()  # Placeholder para lógica de arquivos binários
                                            else:
                                                raise ValueError("Formato de arquivo não suportado")
                                            
                                            for col in config_colunas['colunas']:  
                                                if 'nome' in col and 'tipo' in col:
                                                    if col['tipo'] == 'int':
                                                        df[col['nome']] = df[col['nome']].astype(int)
                                                    elif col['tipo'] == 'float':
                                                        df[col['nome']] = df[col['nome']].astype(float)
                                                    elif col['tipo'] == 'data':
                                                        df[col['nome']] = pd.to_datetime(df[col['nome']], format=col.get('formato', None))
                                                        
                                            banco_caminho = f'/workspaces/projeto_ing/INGESTION/database/{banco}'
                                            conn = sqlite3.connect(banco_caminho)
                                            print(f'Banco de dados: {banco_caminho}')
                                            print(f'Banco de dados criado se não existir: {banco}')
                                            print(f'Conectado!')
                                            print(f"====================================")
                                            
                                            cursor = conn.cursor()
                                            nm_tb = f'tabela_{parte_util}'
                                            sql_create_table = f"CREATE TABLE IF NOT EXISTS {nm_tb} ("
                                            for col in config_colunas['colunas']:
                                                if 'nome' in col and 'tipo' in col:
                                                    sql_create_table += f"{col['nome']} {col['tipo']}, "
                                            sql_create_table = sql_create_table[:-2] + ")"
                                            cursor.execute(sql_create_table)
                                            print(f'Tabela criada: {nm_tb}')
                                            print(f'estrutura da tabela criada: {sql_create_table}')
                                            
                                            for index, row in df.iterrows():
                                                mapped_values = [row.get(col['nome'], None) for col in config_colunas['colunas'] if 'nome' in col]
                                                print("Valores mapeados:", mapped_values)
                                                sql = f"INSERT INTO {nm_tb} ({', '.join([col['nome'] for col in config_colunas['colunas'] if 'nome' in col])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
                                                print("SQL:", sql)
                                                cursor.execute(sql, mapped_values)
                                        
                                            conn.commit()
                                            conn.close()
                                            print("Ingestão de dados concluída com sucesso!")
                                        
                                        # shutil.move(caminho_completo, caminho_backup)
                                        # print(f'Arquivo enviado para backup: {caminho_completo}')
                                        # print(f'Destino: {caminho_backup}')
                                        
                                        
                                    else:
                                        print('Chave não encontrada no arquivo')
                                    
                                    
                                        
                        

item encontrado no diretório: teste
 conteudo: ['estrutura.json', 'tabela_teste2.json', 'tabela_teste.json']
Encontrado estrutura.json em teste
Configuração estrutura: [{'nome_arquivo': 'resultado_teste_', 'descricao': 'tabela teste', 'chave': {'palavra_chave': 'teste', 'banco': 'banco_teste.db', 'tabela': 'tabela_teste'}}, {'nome_arquivo': 'resultado_teste2_', 'descricao': 'tabela teste 2', 'chave': {'palavra_chave': 'teste2', 'banco': 'banco_teste.db', 'tabela': 'tabela_teste2'}}]
{'nome_arquivo': 'resultado_teste_', 'descricao': 'tabela teste', 'chave': {'palavra_chave': 'teste', 'banco': 'banco_teste.db', 'tabela': 'tabela_teste'}}
resultado_teste_
teste
banco_teste.db
tabela_teste
Conteúdo do arquivo de controle de ingestão: ['resultado_teste2_20242306_215405.txt\n']
Procurando em /workspaces/projeto_ing/INGESTION/app/src/landing/...
Verificando arquivo: resultado_teste2_20242306_215405.txt
Arquivo contendo a chave encontrado: /workspaces/projeto_ing/INGESTION/app/src/landing/resu

In [5]:
banco = 'banco_teste.db'
banco_caminho = f'/workspaces/projeto_ing/INGESTION/database/{banco}'
conn = sqlite3.connect(banco_caminho)
cursor = conn.cursor()

query = 'select * from tabela_teste2'

cursor.execute(query)

# Recuperando e imprimindo o resultado
tables = cursor.fetchall()

print("Tabelas no banco de dados:")
for table in tables:
    print(table)

# Fechando a conexão
conn.close()

Tabelas no banco de dados:
(1, 'rafael', 30)
(2, 'joyce', 28)
(1, 'rafael', 30)
(2, 'joyce', 28)
(1, 'rafael', 30)
(2, 'joyce', 28)
(1, 'rafael', 30)
(2, 'joyce', 28)
(1, 'rafael', 30)
(2, 'joyce', 28)
(1, 'rafael', 30)
(2, 'joyce', 28)
(1, 'rafael', 30)
(2, 'joyce', 28)
(1, 'rafael', 30)
(2, 'joyce', 28)
(1, 'rafael', 30)
(2, 'joyce', 28)
(1, 'rafael', 30)
(2, 'joyce', 28)
(1, 'rafael', 30)
(2, 'joyce', 28)


In [53]:
for item in read_layouts:
    accept = os.path.join(layouts, item)
    if os.path.isdir(accept):
        print('========================')
        print(f'item encontrado no diretório: {item}')
        subconteudos = os.listdir(accept)
        print('========================')
        print(f' conteudo: {subconteudos}')
        
        caminho_estrutura_json = os.path.join(accept, 'estrutura.json')
        if os.path.exists(caminho_estrutura_json):
            print(f'Encontrado estrutura.json em {item}')
            
            with open(caminho_estrutura_json, 'r') as f:
                    config_estrutura = json.load(f)
                    print("Configuração estrutura:", config_estrutura)
                    
                    for estr in config_estrutura:
                        nm_arq = estr['nome_arquivo']
                        key = estr['chave']['palavra_chave']
                        banco = estr['chave']['banco']
                        tabela = estr['chave']['tabela']
                        print('=======================')
                        print(estr)
                        print(nm_arq)
                        print(key)
                        print(banco)
                        print(tabela)
                        print('=======================')
                        
                        for root, dirs, lista in os.walk(landing):
                            print(f'Procurando em {root}...')
                            for nome_arquivo in lista:
                                print(f'Verificando arquivo: {nome_arquivo}')
                                if key in nome_arquivo:
                                    caminho_completo = os.path.join(root, nome_arquivo)
                                    print(f'Arquivo contendo a chave encontrado: {caminho_completo}')
                                    sav = os.path.basename(caminho_completo)
                                    print(f'Conteúdo a gravar: {sav}')
                                    
                                    parte_util = sav.split('_')[1]
                                    if parte_util == key:
                                        print('Chave encontrada')
                                        print(f'Separando nome do arquivo: {parte_util}')
                                        caminho_tabela_json = os.path.join(accept, f'{tabela}.json')
                                        print(f'local para criar tabela: {caminho_tabela_json}')
                                        if os.path.exists(caminho_tabela_json):
                                            with open(caminho_tabela_json, 'r') as f:     
                                                config_colunas = json.load(f)
                                            print(config_colunas)
                                        # if os.path.exists(caminho_tabela_json):
                                        #     with open(caminho_tabela_json, 'r') as f:
                                        #         config_colunas = json.load(f)
                                        #     print('Lendo estrutura de coluna json')
                                        #     print(f'Conteúdo da config: {config_colunas}')
                                        #     print(f'Conteúdo do item: {item}')
                                        #     print('==========================')
                                        
                                        
                                    else:
                                        print('Chave não encontrada no arquivo')
                                    
                                    
                                        
                        

item encontrado no diretório: teste
 conteudo: ['estrutura.json', 'tabela_teste2.json', 'tabela_teste.json']
Encontrado estrutura.json em teste
Configuração estrutura: [{'nome_arquivo': 'resultado_teste_', 'descricao': 'tabela teste', 'chave': {'palavra_chave': 'teste', 'banco': 'banco_teste.db', 'tabela': 'tabela_teste'}}, {'nome_arquivo': 'resultado_teste2_', 'descricao': 'tabela teste 2', 'chave': {'palavra_chave': 'teste2', 'banco': 'banco_teste.db', 'tabela': 'tabela_teste_2'}}]
{'nome_arquivo': 'resultado_teste_', 'descricao': 'tabela teste', 'chave': {'palavra_chave': 'teste', 'banco': 'banco_teste.db', 'tabela': 'tabela_teste'}}
resultado_teste_
teste
banco_teste.db
tabela_teste
Procurando em /workspaces/projeto_ing/INGESTION/app/src/landing/...
Verificando arquivo: resultado_teste2_20242306_215405.txt
Arquivo contendo a chave encontrado: /workspaces/projeto_ing/INGESTION/app/src/landing/resultado_teste2_20242306_215405.txt
Conteúdo a gravar: resultado_teste2_20242306_215405.tx